In [1]:
# # Mount the Google drive onto Colab Virtual Environment
# from google.colab import drive
# drive.mount('/content/drive')
root = '.'
# root = '/content/drive/My Drive/English Dataset'

In [2]:
import pandas as pd
import numpy as np
import string
import os
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, Embedding, Dense, CuDNNLSTM, LSTM
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import pickle
from datetime import datetime

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
with open(os.path.join(root, 'en-hi_cleaned.en')) as f:
    lines = f.readlines()
    
lines = pd.DataFrame(lines, columns=['Src'])
lines['Tar'] = None

with open(os.path.join(root, 'en-hi_cleaned.hi')) as f:
    lines['Tar'] = f.readlines()

In [4]:
lines.Src = lines.Src.apply(lambda x: x.strip())
lines.Tar = lines.Tar.apply(lambda x: x.strip())
lines.Tar = '<START> ' + lines.Tar + ' <END>'

In [5]:
lines.shape

(70000, 2)

In [6]:
lines.sample(10)

,Src,Tar
65231,import bookmarks…,<START> पसंद आयात करें <END>
46727,c ase sensitive,<START> स्थिति के प्रति संवेदनशील <END>
29642,properties of s,<START> की विशेषता <END>
12478,interpreter,<START> इंटरफेस <END>
39760,burst mode,<START> बर्स्ट विधि <END>
14676,visible,<START> दृष्टिगोचर <END>
22104,retrieving status…,<START> पुनःप्राप्त कर रहा है स्थिति <END>
27816,could not create context,<START> अस्थायी फाईल बनाने में असमर्थ <END>
9289,gtype,<START> क़िस्म <END>
31732,drag or copy files below to write them to disc,<START> फाइल को नीचे डिस्क में लिखने के लिए खी...


In [7]:
with open(os.path.join(root, 'embeddings.en'), 'rb') as f:
    src_summary = pickle.load(f)
    
with open(os.path.join(root, 'embeddings.hi'), 'rb') as f:
    tar_summary = pickle.load(f)

In [8]:
for word in ['<START>', '<END>']:
    length_of_dict = len(tar_summary['dictionary'])
    tar_summary['dictionary'][word] = length_of_dict
    tar_summary['reverse_dictionary'][length_of_dict] = word
    tar_summary['embeddings'] = np.vstack((tar_summary['embeddings'], np.zeros((tar_summary['embeddings'].shape[1]))))

In [9]:
all_src_words = src_summary['dictionary'].keys()
all_tar_words = tar_summary['dictionary'].keys()

In [10]:
max_len_src = max([len(sentence.split(' ')) for sentence in lines.Src])
max_len_src

134

In [11]:
max_len_tar = max([len(sentence.split(' ')) for sentence in lines.Tar])
max_len_tar

113

In [12]:
num_encoder_tokens = len(all_src_words)
num_decoder_tokens = len(all_tar_words)
(num_encoder_tokens, num_decoder_tokens)

(8333, 4605)

In [13]:
lines = shuffle(lines)
lines.head(10)

,Src,Tar
25443,fetch,<START> पाएँ <END>
33633,blank disc…,<START> खाली डिस्क <END>
18060,create backup files,<START> बैकअप फ़ाइलें बनाएँ <END>
57235,location at date,<START> स्थान तिथि पर <END>
22710,inline the declaration and implementation,<START> इनलाइल डिक्लेरेशन और कार्यान्वयन <END>
25011,path,<START> पथः <END>
20929,source directories,<START> स्रोत निर्देशिकाएँ <END>
46661,font for text with fixed width such as code ex...,<START> कोड उदाहरण के रूप में फिक्स्ड चौड़ाई क...
60504,open image,<START> चित्र खोलें <END>
24837,program interrupt,<START> प्रोग्राम इंटरप्ट <END>


In [14]:
def encode_source(batch_input, batch_size):
    encoder_input_data = np.zeros((batch_size, max_len_src), dtype='float32')
    
    for i, input_text in enumerate(batch_input):
        for t, word in enumerate(input_text.split(' ')):
            if word not in src_summary['dictionary'].keys():
                word = 'UNK'
                
            encoder_input_data[i, t] = src_summary['dictionary'][word]
            
    return encoder_input_data

def encode_target(batch_output, batch_size):
    decoder_input_data = np.zeros((batch_size, max_len_tar), dtype='float32')
    decoder_target_data = np.zeros((batch_size, max_len_tar, num_decoder_tokens), dtype='float32')
    
    for i, target_text in enumerate(batch_output):
        for t, word in enumerate(target_text.split(' ')):
            if word not in tar_summary['dictionary'].keys():
                word = 'UNK'
                
            if t < len(target_text.split())-1:
                decoder_input_data[i, t-1] = tar_summary['dictionary'][word]
                
            if t > 0:
                decoder_target_data[i, t-1, tar_summary['dictionary'][word]] = 1
                
    return decoder_input_data, decoder_target_data

In [15]:
def generate_batch(X, y, batch_size):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = encode_source(X[j:j+batch_size], batch_size)
            decoder_input_data, decoder_target_data = encode_target(y[j:j+batch_size], batch_size)
            
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [16]:
X, y = lines.Src, lines.Tar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X_train.shape, X_test.shape

((63000,), (7000,))

In [17]:
latent_dim = src_summary['embeddings'].shape[1]

In [18]:
encoder_inputs = Input(shape=(None, ), name='Inputs')
enc_emb = Embedding(num_encoder_tokens, latent_dim, mask_zero=True, weights=[src_summary['embeddings']], name='English_Embeddings')(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True, name='Encoder_LSTM')
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

In [19]:
decoder_inputs = Input(shape=(None, ), name='Outputs')
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero=True, weights=[tar_summary['embeddings']], name='Hindi_Embeddings')
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='Decoder_LSTM')
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='Dense_Eng_Hin' + str(datetime.today().date()))
decoder_outputs = decoder_dense(decoder_outputs)

In [20]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [21]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [22]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 32
epochs = 50

In [23]:
filepath = os.path.join(root, "best_model.hdf5")
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [25]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size, 
                    callbacks=callbacks_list)

In [ ]:
model.save(os.path.join(root, 'translator' + str(datetime.today().date()) + '.h5'))

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim, ))
decoder_state_input_c = Input(shape=(latent_dim, ))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = decoder_embedding_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)

In [ ]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tar_summary['dictionary']['<START>']
    
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = tar_summary['reverse_dictionary'][sampled_token_index]
        decoded_sentence += ' ' + sampled_char
        
        if (sampled_char == '<END>') or (len(decoded_sentence) > 50):
            stop_condition = True
            
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        
        states_value = [h, c]
    return decoded_sentence

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
for i in range(10):
    k+=1
    (input_seq, actual_output), _ = next(train_gen)
    decoded_sentence = decode_sequence(input_seq)
    print('Input English sentence:', X_train[k:k+1].values[0])
    print('Actual Hindi Translation:', y_train[k:k+1].values[0][7:-5])
    print('Predicted Hindi Translation:', decoded_sentence[:-5])